In [277]:
import pickle
import re
from collections import defaultdict
import os
import json

In [246]:
g = open('hmm_tagger_l.pickle', 'rb')
m = pickle.load(g)



In [247]:
#taglist
tags = {"NOUN", "VERB", "PROPN", "ADJ", "ADV", "INTJ", \
        "ADP", "AUX", "CCONJ", "SCONJ", "DET", "NUM", "PART", "PRON", \
        "PUNCT", "SYM", "X", \
        "NSUFF", "VSUFF", "AJSUFF", "PREF"}

suff_tag = {"NOUN":"NSUFF", "VERB":"VSUFF", "ADJ":"AJSUFF"}

In [249]:

closed= {}
directory = 'marathi_data/tag_prior_knowledge/closed_class/'
for file in os.listdir(directory):
    if file.endswith(".txt"):
        new_class = set()
        with open(directory+file, 'r') as fn:
            new_class = set(fn.read().split('\n')).difference({"", " ", "\n"})
        closed[file[:-4]] = new_class



In [250]:
indicator= {}
directory = 'marathi_data/tag_prior_knowledge/indicators/'
for file in os.listdir(directory):
    if file.endswith(".txt"):
        new_class = set()
        with open(directory+file, 'r') as fn:
            new_class = set(fn.read().split('\n')).difference({"", " ", "\n"})
        indicator[file[:-4]] = new_class




In [252]:
# indicator["VSUFF"]
# indicator["NSUFF"] = set()
# indicator["AJSUFF"] = set()

In [253]:
def correct_tag(morph, tag, main_tag = None):
    
    if morph.isnumeric():
        closed["NUM"].add(morph)
        return "NUM"
    
    allowed = {"VERB":{"AUX", "ADP", "PART", "ADV"},"NOUN":{"ADP", "PART", "ADV"}, 
               "PRON":{"ADP", "PART", "ADV"}, "ADJ":{"ADP", "PART", "ADV"}, "ADV":{"ADP", "PART", "ADV"},
              "ADP":{"ADP", "PART", "ADV"}}
    for cclass in closed:
        if morph in closed[cclass]:
            if main_tag==None: 
                return cclass
            if main_tag and main_tag not in allowed:
                return cclass
            if main_tag and main_tag in allowed and cclass in allowed[main_tag]:
                return cclass
    
    if len(morph)<=2:
        for cclass in indicator:
            if morph in indicator[cclass]:
                if main_tag==None: 
                    return cclass
                if main_tag and main_tag not in allowed:
                    return cclass
                if main_tag and main_tag in allowed and cclass in allowed[main_tag]:
                    return cclass
        
    if len(morph)>2:    #fuzzy check for equality    
        for cclass in indicator:
            for aff in indicator[cclass]:
                if len(aff) > 2:
                    if morph in aff or aff in morph and abs(len(morph) - len(aff))<=2:
                        if main_tag==None: 
                            return cclass
                        if main_tag and main_tag not in allowed:
                            return cclass
                        if main_tag and main_tag in allowed and cclass in allowed[main_tag]:
                            return cclass
                    
    return tag
                

In [254]:
correct_tag("खाली", "NOUN", main_tag="NOUN")

'ADP'

In [255]:
def update_tag_dist_l(morphs, tags, tag_dist):
    tagged = []
    morph_sent = " ".join(morphs).split(" ")
    for tidx, tag in enumerate(tags):
        morph = morphs[tidx].split(" ")
        
        if len(morph)==1:
            tag = correct_tag(morph[0], tag)
            if tag in closed and morph[0] not in closed[tag]:
                tagged.append("NONE")
                continue
            tag_dist[morph[0]][tag] += 1
            tagged.append(tag)
        if len(morph)>1:
            if morph[0] in indicator["PREF"]:
                tag_dist[morph[0]]["PREF"] += 1
                morph = morph[1:]
                tagged.append("PREF")
                
            if len(morph[0])!=1:
                tag = correct_tag(morph[0], tag)
                if tag in closed and morph[0] not in closed[tag]:
                    tagged+=["NONE"]*len(morph)
                    continue

                tag_dist[morph[0]][tag] += 1
                morph = morph[1:]
                tagged.append(tag)
            
                for midx, m in enumerate(morph):
                    if correct_tag(m, tag, tag) != tag: #if the morph is in the closed classes/indicators
                        tag_dist[m][correct_tag(m, tag, tag)] += 1
                        tagged.append(correct_tag(m, tag, tag))
                    else: #otherwise, we put in suffix bucket
                        if tag in suff_tag:
                            tag_dist[m][suff_tag[tag]]+=1
                            tagged.append(suff_tag[tag])
                        else:
                            tag_dist[m][tag] += 1
                            tagged.append(tag)
#     print("\n\nTAGGED")
#     print(*list(zip(morph_sent, tagged)), sep = "\n")
                
    return tag_dist
                

In [256]:
def get_main_morph(tok):
#     print(tok)
    for aidx, a in enumerate(tok.split(" ")):
        if a not in indicator["PREF"] and len(a)>1:
#             print(aidx)
            return aidx
    
    return 0
    

In [258]:
#unlabelled segmented data
data = open("marathi_data/mr.segmented.EOLmarked.txt", "r").read().split("E O L\n")
sentences = [re.split(' |\n', ss)[:-1] for ss in data][:-1]
morphs = [sent.split("\n")[:-1] for sent in data] #token with morphs

In [259]:
tag_dist = defaultdict(lambda: defaultdict(lambda: 0))
i = 0
for sidx, sent in enumerate(sentences):
    sidx += i
    all_tags = m.best_path(sent)
#     print(len(sent))
#     print(sent)
    tag_ids = []
    tidx = 0
#     print(*morphs[sidx], sep="\n")
    for tok in morphs[sidx]:
        main_id = get_main_morph(tok)
        tag_ids.append(tidx+main_id)
#         print(tag_ids)
        tidx += len(tok.split(" "))
    tags = [all_tags[tidx] for tidx in tag_ids]
#     print("INITIAL")
#     print(*list(zip(morphs[sidx], tags)), sep="\n")
    assert(len(tags) == len(morphs[sidx]))
    tag_dist = update_tag_dist_l(morphs[sidx], tags, tag_dist)



In [163]:
s = "खबर"
tag_dist[s]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'NOUN': 7, 'X': 2, 'ADV': 1, 'PROPN': 1, 'DET': 1})

In [260]:
token_dist = defaultdict(lambda: defaultdict(lambda: 0))
for token in tag_dist:
    for tag in tag_dist[token]:
        token_dist[tag][token] = tag_dist[token][tag]

In [121]:
indicator["VSUFF"]

set()

In [262]:
sorted(list(token_dist["NOUN"].items()), key = lambda x: x[1], reverse = True)[:200]

[('लोक', 1442),
 ('आज', 1169),
 ('माहिती', 1082),
 ('पोलिस', 1019),
 ('वाढ', 1010),
 ('विचार', 911),
 ('लक्ष', 716),
 ('केंद्र', 714),
 ('प्रकार', 674),
 ('चित्रपट', 665),
 ('भाग', 631),
 ('मुख्यमंत्र', 616),
 ('अधिकार', 606),
 ('घर', 603),
 ('अर्थ', 603),
 ('प्रश्न', 564),
 ('कार्य', 562),
 ('मार्ग', 561),
 ('दिवस', 522),
 ('सध्या', 520),
 ('मागणी', 516),
 ('हात', 510),
 ('अध्यक्ष', 497),
 ('गाव', 486),
 ('धाव', 464),
 ('पक्ष', 462),
 ('वर्षां', 458),
 ('व्यक्ती', 455),
 ('निर्माण', 451),
 ('संघा', 427),
 ('रुपय', 426),
 ('नव्हत', 425),
 ('शिक्षण', 424),
 ('महत्त्व', 418),
 ('कंपनी', 413),
 ('सुमार', 410),
 ('कार्यक्रम', 409),
 ('खूप', 404),
 ('मुला', 397),
 ('सरकार', 393),
 ('विभागा', 391),
 ('परिसर', 384),
 ('गुन्ह', 383),
 ('मोठ', 380),
 ('शोध', 374),
 ('मोदी', 369),
 ('सकाळ', 361),
 ('दिवसां', 341),
 ('जोर', 339),
 ('विषय', 335),
 ('उभ', 330),
 ('राज्य', 323),
 ('पाण्या', 317),
 ('चित्र', 316),
 ('मांड', 313),
 ('क्षेत्र', 309),
 ('परिस्थिती', 308),
 ('विजय', 308),
 ('आप', 301),
 

In [165]:
sorted(list(token_dist["VSUFF"].items()), key = lambda x: x[1], reverse = True)[:200]

[('त', 7060),
 ('ून', 3891),
 ('ले', 3292),
 ('आ', 2284),
 ('ली', 2097),
 ('ण्यासाठी', 1764),
 ('लेल्या', 1466),
 ('ू', 1387),
 ('णे', 1171),
 ('ता', 1095),
 ('लं', 1032),
 ('ते', 931),
 ('ण्यात', 783),
 ('लेले', 731),
 ('ने', 694),
 ('तो', 656),
 ('लेली', 544),
 ('ताना', 539),
 ('ी', 534),
 ('ल्या', 518),
 ('ावी', 517),
 ('तील', 484),
 ('ीत', 469),
 ('े', 431),
 ('णार', 408),
 ('ण्याची', 396),
 ('ं', 392),
 ('ल्यानंतर', 372),
 ('णाऱ्या', 366),
 ('ण्यास', 355),
 ('णं', 319),
 ('ण्याचा', 302),
 ('णारे', 281),
 ('साठी', 280),
 ('ल्याने', 263),
 ('चं', 254),
 ('ुन', 249),
 ('णारा', 235),
 ('ा', 223),
 ('ावा', 215),
 ('मुळे', 200),
 ('णार्या', 181),
 ('णारी', 181),
 ('य', 175),
 ('ण्याचे', 169),
 ('लो', 150),
 ('वळ', 141),
 ('ण्या', 141),
 ('गामी', 119),
 ('ायचं', 119),
 ('ावं', 101),
 ('ताच', 97),
 ('वून', 91),
 ('हाण', 89),
 ('तं', 87),
 ('लेलं', 86),
 ('णी', 86),
 ('’', 85),
 ('नामा', 84),
 ('ंना', 83),
 ('असता', 80),
 ('से', 79),
 ('रल', 78),
 ('नंतर', 71),
 ('ः', 69),
 ('च', 68),
 ('प

In [105]:
sorted(list(token_dist["NSUFF"].items()), key = lambda x: x[1], reverse = True)[:200]

[('ी', 8393),
 ('े', 6859),
 ('ा', 6262),
 ('त', 1668),
 ('ं', 1541),
 ('ने', 1198),
 ('ित', 942),
 ('ांना', 874),
 ('ाने', 846),
 ('ासाठी', 602),
 ('ली', 596),
 ('दार', 579),
 ('त्यात', 571),
 ('सध्या', 536),
 ('ीय', 496),
 ('ून', 488),
 ('साठी', 458),
 ('ीत', 408),
 ('मंत्री', 389),
 ('त्याच', 373),
 ('ास', 351),
 ('पासून', 347),
 ('ांत', 329),
 ('तील', 311),
 ('्यांनी', 282),
 ('्यांना', 278),
 ('री', 274),
 ('ले', 262),
 ('ामुळे', 257),
 ('नव्या', 251),
 ('ांसाठी', 249),
 ('ता', 247),
 ('कडे', 235),
 ('िक', 219),
 ('पूर्वी', 215),
 ('न्याय', 206),
 ('ील', 201),
 ('मुळे', 200),
 ('द्याव', 195),
 ('ू', 195),
 ('ंना', 179),
 ('वून', 178),
 ('ाचं', 177),
 ('ांकडून', 167),
 ('ाकडे', 167),
 ('्याच्या', 164),
 ('ार्थ', 158),
 ('तः', 158),
 ('च', 151),
 ('स', 150),
 ('चं', 150),
 ('ात्मक', 149),
 ('ावरील', 138),
 ('ण्यासाठी', 137),
 ('नुसार', 136),
 ('्यावर', 134),
 ('ापासून', 133),
 ('ते', 131),
 ('लं', 129),
 ('कडून', 128),
 ('पर्यंत', 127),
 ('व्या', 126),
 ('मान', 123),
 ('नंतर', 121),

In [ ]:
sorted(list(token_dist["VSUFF"].items()), key = lambda x: x[1], reverse = True)[:200]

In [196]:
data = open("marathi_data/mr.segmented.EOLmarked.txt", "r")


In [197]:
maxl = 0
i=0
count = 0
for line in data:
    if len(line.split(" ")) > 2:
        count+=1
#         print(line)
    maxl =  max(maxl, len(line.split(" ")))

In [198]:
count

82256

In [ ]:
#EVALUATION

In [200]:
import stanza
nlp = stanza.Pipeline('mr', processors='tokenize,mwt,pos')

2021-07-29 16:19:14 INFO: Loading these models for language: mr (Marathi):
| Processor | Package |
-----------------------
| tokenize  | ufal    |
| mwt       | ufal    |
| pos       | ufal    |

2021-07-29 16:19:14 INFO: Use device: cpu
2021-07-29 16:19:14 INFO: Loading: tokenize
2021-07-29 16:19:16 INFO: Loading: mwt
2021-07-29 16:19:16 INFO: Loading: pos
2021-07-29 16:19:17 INFO: Done loading processors!


In [228]:
best = 0
second_best = 0
i = 0
prob5 = 0
total = 1000
for morph in tag_dist:
    if i>total:
        break
    if len(morph)<3:
        continue
    doc = nlp(morph)
    tag = doc.sentences[0].words[0].upos
    if tag not in ["VERB"]:
        continue
    if tag not in ["NOUN", "VERB", "ADJ", "ADV", "PRON", "DET", "ADP", "PROPN", "PART"]:
        continue
    i+=1
    tag_rank = sorted(list(tag_dist[morph].items()), key = lambda x: x[1], reverse=True)
    print(morph)
    print(tag)
    print(tag_rank)
    print("\n\n\n")
    
    if tag_rank[0][0]==tag:
        best+=1
    if len(tag_rank)>1:
        if tag_rank[1][0]==tag:
            second_best+=1
            
    if tag not in tag_dist[morph]:
        continue
    
    tot = sum([v for k,v in tag_dist[morph].items()])
    perc = float(tag_dist[morph][tag]/tot)
    if perc > 0.5:
        prob5 +=1
    
    

देणार
VERB
[('NOUN', 118), ('VERB', 73), ('VSUFF', 22), ('X', 1)]




ण्याची
VERB
[('VSUFF', 396), ('NSUFF', 38), ('ADP', 22), ('ADV', 2), ('AUX', 1), ('SCONJ', 1), ('X', 1)]




झाला
VERB
[('ADP', 1085)]




येणार
VERB
[('VERB', 541), ('NOUN', 108)]




पहा
VERB
[('VERB', 71), ('X', 19)]




मावळ
VERB
[('NOUN', 11), ('PROPN', 9), ('ADV', 8), ('VERB', 7), ('VSUFF', 1)]




्यांवर
VERB
[('NSUFF', 41), ('ADP', 2), ('VSUFF', 2), ('X', 1), ('ADV', 1)]




करण्या
VERB
[('VERB', 3219), ('VSUFF', 1)]




आदेश
VERB
[('ADV', 157), ('NOUN', 77), ('PROPN', 4)]




दिल्या
VERB
[('VERB', 260), ('ADV', 13)]




सांग
VERB
[('VERB', 438), ('PROPN', 5), ('NOUN', 4), ('NSUFF', 2), ('X', 1)]




चालू
VERB
[('ADP', 217), ('AJSUFF', 1)]




ातील
VERB
[('ADP', 1218), ('PROPN', 51), ('X', 42), ('AJSUFF', 9), ('ADV', 3), ('AUX', 2)]




असल्या
VERB
[('VERB', 2058)]




घडली
VERB
[('VERB', 156), ('PROPN', 1), ('X', 1), ('NSUFF', 1)]




आले
VERB
[('X', 1485)]




क्यात
VERB
[('NSUFF', 66), ('ADP', 1), ('AJSUFF

ीतील
VERB
[('NSUFF', 26), ('AJSUFF', 2), ('VSUFF', 2), ('PROPN', 1), ('NUM', 1)]




ांसोबत
VERB
[('NSUFF', 45), ('PROPN', 3), ('AJSUFF', 1)]




दिस
VERB
[('VERB', 438)]




शोध
VERB
[('NOUN', 374), ('AJSUFF', 8), ('NSUFF', 5)]




ायला
VERB
[('ADP', 760), ('X', 5), ('PROPN', 4), ('SCONJ', 1), ('NUM', 1)]




जात
VERB
[('AUX', 1057), ('ADV', 5), ('AJSUFF', 4), ('NUM', 2), ('PROPN', 1)]




दीपिका
VERB
[('PROPN', 8), ('ADV', 3)]




समावेश
VERB
[('VERB', 181), ('ADJ', 100), ('X', 8), ('AJSUFF', 6), ('PROPN', 1)]




व्हा
VERB
[('AJSUFF', 166), ('NSUFF', 44), ('PRON', 18), ('ADJ', 7), ('X', 3), ('PROPN', 1)]




करून
VERB
[('NSUFF', 120), ('VSUFF', 63), ('X', 5), ('ADV', 3), ('AJSUFF', 3), ('ADP', 2), ('PART', 1)]




राहिल
VERB
[('NOUN', 142), ('VERB', 41)]




सूर
VERB
[('X', 24), ('ADJ', 14), ('ADV', 13), ('PROPN', 3), ('VERB', 2), ('AUX', 1), ('ADP', 1)]




देणे
VERB
[('VERB', 158)]




धडक
VERB
[('ADV', 26), ('VERB', 20), ('NOUN', 13)]




ठार
VERB
[('ADV', 18), ('NSUFF', 11), ('X

राख
VERB
[('VERB', 123), ('X', 4), ('NOUN', 4)]




लागल्या
VERB
[('VERB', 51)]




ण्याचा
VERB
[('VSUFF', 302), ('NSUFF', 34), ('ADP', 18), ('X', 4), ('ADV', 2)]




आठवड्या
VERB
[('NOUN', 106), ('DET', 1)]




चाल
VERB
[('ADP', 388), ('ADV', 3), ('AJSUFF', 2), ('NSUFF', 1), ('PROPN', 1)]




आकार
VERB
[('ADJ', 85), ('NOUN', 83)]




करू
VERB
[('VERB', 370), ('NSUFF', 22), ('AJSUFF', 9), ('VSUFF', 4), ('ADP', 1)]




शकतो
VERB
[('AUX', 273)]




व्हो
VERB
[('X', 8), ('PROPN', 7), ('ADP', 1)]




मेल
VERB
[('X', 17), ('PROPN', 13), ('ADJ', 7), ('VSUFF', 4), ('NOUN', 3), ('PREF', 2), ('AJSUFF', 1), ('PRON', 1)]




दिशा
VERB
[('NOUN', 82), ('PROPN', 1), ('AJSUFF', 1)]




करणार
VERB
[('VERB', 335)]




कॉल
VERB
[('X', 22), ('ADV', 19), ('VSUFF', 6), ('ADJ', 6), ('PROPN', 3), ('NSUFF', 2), ('PUNCT', 1)]




दिव्यांग
VERB
[('NOUN', 7), ('AJSUFF', 3), ('DET', 1), ('ADP', 1)]




कल्याण
VERB
[('PROPN', 79), ('NOUN', 40), ('NSUFF', 15), ('AJSUFF', 2)]




पास
VERB
[('ADP', 140), ('ADV', 84),

राडे
VERB
[('X', 8), ('PROPN', 4), ('PART', 1)]




सवरेत्
VERB
[('AJSUFF', 1), ('ADJ', 1)]




आजमावत
VERB
[('VERB', 3), ('NOUN', 2)]




निघालो
VERB
[('VERB', 13)]




घ्या
VERB
[('AUX', 223), ('ADV', 31), ('NSUFF', 5), ('AJSUFF', 5)]




आधी
VERB
[('ADV', 236), ('NSUFF', 18), ('VSUFF', 11), ('PRON', 11), ('AJSUFF', 4), ('ADP', 3), ('PUNCT', 1), ('X', 1)]




बुडाले
VERB
[('VERB', 6), ('NOUN', 1)]




सोल
VERB
[('X', 31), ('NOUN', 5), ('PROPN', 2), ('VERB', 2)]




काटा
VERB
[('NSUFF', 7), ('X', 2), ('ADV', 1), ('VERB', 1)]




घेतले
VERB
[('VERB', 369), ('ADV', 7), ('X', 1)]




दिल
VERB
[('NOUN', 79), ('VERB', 61), ('AJSUFF', 2), ('ADP', 2), ('NUM', 2)]




घेतल्या
VERB
[('VERB', 85), ('X', 1)]




ेबंदी’
VERB
[('VSUFF', 1)]




रहीम
VERB
[('PROPN', 8)]




सुनावण्या
VERB
[('VERB', 11), ('ADP', 1)]




नोट
VERB
[('NOUN', 91), ('PROPN', 8), ('AJSUFF', 1)]




आदरांजली
VERB
[('ADV', 5), ('VERB', 1)]




ब्राउझ
VERB
[('X', 7), ('AJSUFF', 3)]




गूड
VERB
[('AJSUFF', 2), ('VSUFF', 2)]


ऋतु
VERB
[('PROPN', 7), ('NSUFF', 1), ('VERB', 1)]




आयोजक
VERB
[('NOUN', 4), ('VERB', 2), ('PROPN', 2), ('ADV', 1), ('AJSUFF', 1), ('X', 1)]




जीवी
VERB
[('AJSUFF', 7), ('PROPN', 3)]




ten
VERB
[('PROPN', 5), ('X', 1)]




सापडत
VERB
[('VERB', 14), ('NOUN', 11), ('X', 3)]




सहकाऱ
VERB
[('NOUN', 20)]




आजी
VERB
[('NOUN', 26), ('PROPN', 14), ('VERB', 9), ('X', 6), ('ADP', 4)]




उंच
VERB
[('ADJ', 21), ('X', 16), ('NOUN', 10), ('VERB', 8)]




आकाश
VERB
[('NOUN', 35), ('X', 7), ('PROPN', 4), ('NSUFF', 2), ('AJSUFF', 2)]




पाहा
VERB
[('VERB', 43), ('PROPN', 6), ('X', 5)]




विकास
VERB
[('NOUN', 145), ('ADJ', 114), ('PROPN', 70), ('NSUFF', 54), ('ADV', 27), ('AJSUFF', 14)]




उचल
VERB
[('VERB', 122), ('AJSUFF', 1)]




चौथ
VERB
[('NOUN', 62), ('VSUFF', 1)]




रांगे
VERB
[('VERB', 12), ('NOUN', 2), ('PROPN', 1), ('NSUFF', 1), ('X', 1)]




क्रम
VERB
[('ADJ', 36), ('VERB', 17), ('AJSUFF', 7), ('NSUFF', 4), ('ADV', 3), ('VSUFF', 2), ('X', 2), ('PRON', 1)]




डेकर
VERB
[('PROP

आयडी
VERB
[('PROPN', 10), ('X', 1)]




वर्ड
VERB
[('ADV', 9), ('ADP', 6), ('CCONJ', 3), ('X', 3), ('AJSUFF', 1), ('PROPN', 1)]




ज्योत
VERB
[('AJSUFF', 14), ('X', 2), ('ADV', 1)]




देऊ
VERB
[('VERB', 73), ('NSUFF', 5), ('ADV', 2)]




शिवराय
VERB
[('PROPN', 11), ('NOUN', 6), ('AJSUFF', 1)]




सोप
VERB
[('NOUN', 54), ('X', 1)]




मोडी
VERB
[('NSUFF', 4), ('ADV', 3), ('VSUFF', 1), ('PROPN', 1)]




11.
VERB
[('NUM', 1)]




घ्यावे
VERB
[('AUX', 46)]




गणेशोत्सव
VERB
[('NOUN', 34), ('ADV', 3), ('PROPN', 2), ('AJSUFF', 1)]




बाजारपेठ
VERB
[('NOUN', 61), ('VERB', 1)]




होल
VERB
[('X', 9), ('PROPN', 5)]




घाला
VERB
[('ADP', 31)]




येऊन
VERB
[('ADV', 30)]




पिया
VERB
[('X', 4), ('VERB', 2)]




आलीय
VERB
[('AUX', 24)]




किल
VERB
[('PROPN', 7), ('ADV', 3)]




लोणावळ
VERB
[('NOUN', 13)]




लेण्यां
VERB
[('NOUN', 2)]




वयास
VERB
[('VSUFF', 22), ('NSUFF', 3), ('AUX', 1), ('X', 1)]




लाय
VERB
[('AUX', 155), ('ADP', 14), ('X', 7), ('PROPN', 6), ('AJSUFF', 1), ('NSUFF', 1)

द्रावक
VERB
[('AJSUFF', 3)]




‘बिचारं’
VERB
[('ADV', 1)]




चाकू
VERB
[('ADP', 21)]




फिरत
VERB
[('VERB', 12), ('NOUN', 3), ('X', 2), ('NSUFF', 2), ('ADV', 1)]




टेबल
VERB
[('NOUN', 21), ('PROPN', 14), ('VERB', 1), ('ADP', 1), ('NSUFF', 1), ('ADV', 1)]




फोंडा
VERB
[('PROPN', 7), ('NSUFF', 1), ('ADV', 1)]




सीट
VERB
[('PROPN', 5), ('NOUN', 1)]




शुभेच्छा
VERB
[('ADV', 28), ('VERB', 5), ('PROPN', 4)]




पाहता
VERB
[('ADJ', 41), ('X', 39), ('NSUFF', 35)]




माते
VERB
[('ADP', 38), ('AJSUFF', 1), ('ADV', 1), ('PROPN', 1)]




खावे
VERB
[('X', 4), ('NSUFF', 1), ('ADP', 1)]




लोप
VERB
[('X', 3), ('PROPN', 2)]




एवढय़ा
VERB
[('VERB', 2), ('ADP', 1)]




धमकाव
VERB
[('VERB', 13)]




पदाधिकार
VERB
[('NOUN', 100)]




निरसन
VERB
[('ADV', 2), ('NOUN', 1)]




मागणार
VERB
[('NOUN', 5), ('VERB', 2)]




घेणे
VERB
[('NSUFF', 24), ('AJSUFF', 4), ('VERB', 3), ('X', 2), ('VSUFF', 2)]




हेतु
VERB
[('ADJ', 6), ('NSUFF', 2), ('X', 1)]




सॉफ्टवेअर
VERB
[('PROPN', 8), ('AJSUFF', 4),

एक्स्प्रेस
VERB
[('PROPN', 12)]




ठाण्या
VERB
[('VERB', 6), ('NOUN', 2)]




लाचखोर
VERB
[('ADJ', 2), ('NOUN', 1)]




लिए
VERB
[('VSUFF', 14), ('X', 2)]




शॉर्ट
VERB
[('X', 3), ('VERB', 1)]




साथी
VERB
[('AJSUFF', 1), ('ADP', 1), ('VERB', 1)]




आंबे
VERB
[('X', 14), ('ADJ', 13), ('PROPN', 2), ('ADP', 1), ('DET', 1)]




जोगा
VERB
[('VSUFF', 7), ('PROPN', 5), ('NSUFF', 1)]




सहकार्य
VERB
[('ADV', 32), ('NOUN', 10), ('NSUFF', 1)]




आक्षेप
VERB
[('ADV', 27), ('AJSUFF', 7), ('NOUN', 2), ('NSUFF', 1)]




आणणार
VERB
[('NOUN', 27), ('VERB', 5)]




इन्
VERB
[('X', 13), ('PROPN', 3)]




क्लेम
VERB
[('X', 1)]




एसी
VERB
[('PROPN', 12), ('ADV', 1), ('VERB', 1)]




लोकल
VERB
[('PROPN', 16)]




टूर
VERB
[('PROPN', 21), ('ADJ', 1)]




वावे
VERB
[('AUX', 14), ('NSUFF', 2), ('AJSUFF', 1), ('X', 1), ('ADP', 1)]




चारचौघ
VERB
[('NOUN', 5), ('PROPN', 1)]




ट्रेलर
VERB
[('ADJ', 14), ('X', 7), ('ADV', 4)]




यूज
VERB
[('NOUN', 6)]




सौदागर
VERB
[('NOUN', 6), ('NSUFF', 1)]




शा

In [230]:
best

169

148

In [201]:
doc = nlp('पीण्यासाठी')
tag = doc.sentences[0].words[0].upos

In [205]:
morph = "घर"
best_tag = sorted(list(tag_dist[morph].items()), key = lambda x: x[1], reverse=True)[0][0]

In [206]:
best_tag

'NOUN'

In [273]:
tag_dist["प्रभाव"]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'X': 7, 'NOUN': 204, 'ADJ': 3, 'NSUFF': 1, 'AJSUFF': 1, 'ADV': 1})

In [274]:
len(tag_dist)

16688

In [278]:
with open("tag_distributions/tag_dist_l.json", "w") as l_file:
    json.dump(tag_dist, l_file, ensure_ascii=False)

In [ ]:
#EVALUATION

In [271]:
best = 0
second_best = 0
i = 0
prob5 = 0
total = 0
admissible = 0

f = open("marathi_data/top2000", "r").read().split("\n")
for line in f:
    total+=1
    if total<1200:
        continue
    if len(line.split())<=1:
        continue
    morph = line.strip().split(" ")[1]

    
#     if len(morph)<3:
#         continue
#     doc = nlp(morph)
#     tag = doc.sentences[0].words[0].upos
#     if tag not in ["NOUN"]:
#         continue
#     if tag not in ["NOUN", "VERB", "ADJ", "ADV", "PRON", "DET", "ADP", "PROPN", "PART"]:
#         continue
    
    tag_rank = sorted(list(tag_dist[morph].items()), key = lambda x: x[1], reverse=True)
    print(morph)
#     print(tag)
    print(tag_rank)
    print("\n\n\n")
    if tag_rank[0][0]=="PUNCT":
        continue
    i+=1
    
    print("NUMBER ", i)
#     if len(tag_rank)==0:
#         continue
#     if tag_rank[0][0]==tag:
#         best+=1
#     if len(tag_rank)>1:
#         if tag_rank[1][0]==tag:
#             second_best+=1
            
#     if tag not in tag_dist[morph]:
#         continue
    
    ask = input()
    if ask=="1":
        admissible += 1
    
#     tot = sum([v for k,v in tag_dist[morph].items()])
#     perc = float(tag_dist[morph][tag]/tot)
#     if perc > 0.5:
#         prob5 +=1
    
    

पुरुष
[('NOUN', 38), ('PROPN', 28), ('ADJ', 27)]




NUMBER  1
1
पत्नी
[('ADJ', 85), ('AJSUFF', 3), ('X', 1), ('PROPN', 1), ('NSUFF', 1)]




NUMBER  2
2
नका
[('AUX', 174)]




NUMBER  3
2
तिने
[('PRON', 172)]




NUMBER  4
1
ा
[('NSUFF', 6522), ('AJSUFF', 334), ('VSUFF', 229), ('X', 44), ('PROPN', 31), ('ADV', 14), ('AUX', 11), ('ADP', 11), ('NUM', 4), ('PART', 3), ('CCONJ', 1), ('PRON', 1)]




NUMBER  5
1
स्वच्छ
[('X', 62), ('VERB', 50), ('NSUFF', 10), ('ADP', 2), ('ADJ', 2), ('PROPN', 1)]




NUMBER  6
2
निर्
[('NOUN', 104), ('ADV', 24), ('X', 10), ('NSUFF', 7), ('ADP', 2)]




NUMBER  7
2
.
[('PUNCT', 58511)]




१२
[('NUM', 167)]




NUMBER  8
1
सारखे
[('ADV', 139)]




NUMBER  9
1
साज
[('X', 50), ('NOUN', 30), ('ADV', 1)]




NUMBER  10
1
पात्र
[('ADJ', 85), ('NOUN', 72), ('NSUFF', 2), ('X', 2), ('AJSUFF', 1), ('VSUFF', 1)]




NUMBER  11
1
अंदाज
[('NOUN', 49), ('ADV', 41), ('VERB', 19)]




NUMBER  12
1
:
[('PUNCT', 1274)]




.
[('PUNCT', 58511)]




े
[('NSUFF', 6922), ('VSUF

KeyboardInterrupt: Interrupted by user

In [265]:
admissible

182

In [269]:
admissible

164

In [272]:
admissible

34